In [1]:
import pandas as pd
import numpy as np
import time
from datetime import timedelta 
from datetime import datetime

In [2]:
eb = pd.read_csv('78 EB.csv')

In [3]:
eb

,Date,Time,Id,Route,Direction,Name,MlDetPostmile,pcw1occ,pcw1speed,pcw1vol,r_1_offcnt,r_2_offcnt,Psg 1 Cnt,Psg 2 Cnt,Psg 3 Cnt,Mainline Lanes
0,2021/02/11,05:30:00,344,78,EB,78 EB W/O Jefferson St,0.4150,1.6,61,213,NaN,NaN,NaN,NaN,NaN,0xF0
1,2021/02/11,05:30:00,201,78,EB,Jefferson St to 78 EB,0.8580,1.4,73,213,0.0,0.0,0.0,0.0,0.0,0xE0
2,2021/02/11,05:30:00,200,78,EB,El Camino Real to 78 EB,1.5940,2.5,63,335,1.0,NaN,3.0,3.0,NaN,0xE0
3,2021/02/11,05:30:00,470,78,EB,78 EB @ College Blvd,3.2235,3.7,54,426,NaN,NaN,0.0,2.0,0.0,0xFC
4,2021/02/11,05:30:00,199,78,EB,Plaza Dr to 78 EB,3.5860,2.1,59,274,0.0,0.0,0.0,0.0,NaN,0xFC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16815,2021/02/11,19:30:00,13006,78,EB,Nordahl Rd to 78 EB,15.5960,4.5,66,639,1.0,2.0,7.0,9.0,NaN,0xE0
16816,2021/02/11,19:30:00,13015,78,EB,78 EB E/O Nordahl,15.9155,7.1,56,853,NaN,NaN,NaN,NaN,NaN,0xF0
16817,2021/02/11,19:30:00,13014,78,EB,78 EB W/O 15,16.2677,4.0,57,487,NaN,NaN,NaN,NaN,NaN,0xE0
16818,2021/02/11,19:30:00,23014,78,EB,15 Connector from 78 EB,16.2683,4.0,57,487,NaN,NaN,NaN,NaN,NaN,0xC0


In [4]:
eb = eb[eb.Name.str.contains('to')]

In [5]:
eb

,Date,Time,Id,Route,Direction,Name,MlDetPostmile,pcw1occ,pcw1speed,pcw1vol,r_1_offcnt,r_2_offcnt,Psg 1 Cnt,Psg 2 Cnt,Psg 3 Cnt,Mainline Lanes
1,2021/02/11,05:30:00,201,78,EB,Jefferson St to 78 EB,0.8580,1.4,73,213,0.0,0.0,0.0,0.0,0.0,0xE0
2,2021/02/11,05:30:00,200,78,EB,El Camino Real to 78 EB,1.5940,2.5,63,335,1.0,NaN,3.0,3.0,NaN,0xE0
4,2021/02/11,05:30:00,199,78,EB,Plaza Dr to 78 EB,3.5860,2.1,59,274,0.0,0.0,0.0,0.0,NaN,0xFC
5,2021/02/11,05:30:00,24,78,EB,Emerald Dr to 78 EB,4.4740,3.0,52,335,0.0,0.0,0.0,0.0,NaN,0xFC
6,2021/02/11,05:30:00,205,78,EB,Vista Village Dr to 78 EB,6.3200,3.0,79,517,0.0,0.0,0.0,4.0,1.0,0xE0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16812,2021/02/11,19:30:00,180,78,EB,San Marcos Blvd to 78 EB,12.2740,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0xE0
16813,2021/02/11,19:30:00,234,78,EB,Twin Oaks Vlly Rd to 78 EB,13.0220,6.2,64,853,1.0,3.0,0.0,2.0,2.0,0xE0
16814,2021/02/11,19:30:00,236,78,EB,Barham/Woodland to 78 EB,14.8600,5.9,67,853,NaN,NaN,0.0,1.0,NaN,0xFC
16815,2021/02/11,19:30:00,13006,78,EB,Nordahl Rd to 78 EB,15.5960,4.5,66,639,1.0,2.0,7.0,9.0,NaN,0xE0


In [6]:
wb = pd.read_csv('78 WB .csv')

In [7]:
wb = wb[wb.Name.str.contains('to')]

In [8]:
wb.head()

,Date,Time,Id,Route,Direction,Name,MlDetPostmile,pcw1occ,pcw1speed,pcw1vol,r_1_offcnt,r_2_offcnt,Psg 1 Cnt,Psg 2 Cnt,Psg 3 Cnt,Mainline Lanes
1,2021/02/11,05:30:00,190,78,WB,Jefferson St to 78 WB,0.653,3.6,64,487,0.0,1.0,0.0,0.0,0.0,0xE0
2,2021/02/11,05:30:00,191,78,WB,El Camino Real to 78 WB,1.387,3.8,64,517,0.0,1.0,1.0,2.0,NaN,0xE0
3,2021/02/11,05:30:00,66,78,WB,College Blvd SB to 78 WB,3.245,3.7,54,426,NaN,NaN,0.0,2.0,0.0,0xFC
4,2021/02/11,05:30:00,270,78,WB,College Blvd NB to 78 WB,3.330,4.0,60,517,1.0,2.0,0.0,1.0,NaN,0xE0
6,2021/02/11,05:30:00,192,78,WB,Emerald Dr to 78 WB,4.285,3.9,69,578,NaN,NaN,0.0,0.0,NaN,0xE0


In [9]:
wb['Time'].unique()

array(['05:30:00', '05:31:00', '05:32:00', '05:33:00', '05:34:00',
       '05:35:00', '05:36:00', '05:37:00', '05:38:00', '05:39:00',
       '05:40:00', '05:41:00', '05:42:00', '05:43:00', '05:44:00',
       '05:45:00', '05:46:00', '05:47:00', '05:48:00', '05:49:00',
       '05:50:00', '05:51:00', '05:52:00', '05:53:00', '05:54:00',
       '05:55:00', '05:56:00', '05:57:00', '05:58:00', '05:59:00',
       '06:00:00', '06:01:00', '06:02:00', '06:03:00', '06:04:00',
       '06:05:00', '06:06:00', '06:07:00', '06:08:00', '06:09:00',
       '06:10:00', '06:11:00', '06:12:00', '06:13:00', '06:14:00',
       '06:15:00', '06:16:00', '06:17:00', '06:18:00', '06:19:00',
       '06:20:00', '06:21:00', '06:22:00', '06:23:00', '06:24:00',
       '06:25:00', '06:26:00', '06:27:00', '06:28:00', '06:29:00',
       '06:30:00', '06:31:00', '06:32:00', '06:33:00', '06:34:00',
       '06:35:00', '06:36:00', '06:37:00', '06:38:00', '06:39:00',
       '06:40:00', '06:41:00', '06:42:00', '06:43:00', '06:44:

In [10]:
eb["Time"].unique()[0]

'05:30:00'

In [11]:
def convtime(s):
    fmt = "%H:%M:%S"
    return datetime.strptime(s, fmt)

In [12]:
wb['wb_convtime'] = wb['Time'].apply(convtime)
wb['delay_time'] = wb['wb_convtime'] + timedelta(minutes=1)  

In [13]:
wb

,Date,Time,Id,Route,Direction,Name,MlDetPostmile,pcw1occ,pcw1speed,pcw1vol,r_1_offcnt,r_2_offcnt,Psg 1 Cnt,Psg 2 Cnt,Psg 3 Cnt,Mainline Lanes,wb_convtime,delay_time
1,2021/02/11,05:30:00,190,78,WB,Jefferson St to 78 WB,0.653,3.6,64,487,0.0,1.0,0.0,0.0,0.0,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00
2,2021/02/11,05:30:00,191,78,WB,El Camino Real to 78 WB,1.387,3.8,64,517,0.0,1.0,1.0,2.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00
3,2021/02/11,05:30:00,66,78,WB,College Blvd SB to 78 WB,3.245,3.7,54,426,NaN,NaN,0.0,2.0,0.0,0xFC,1900-01-01 05:30:00,1900-01-01 05:31:00
4,2021/02/11,05:30:00,270,78,WB,College Blvd NB to 78 WB,3.330,4.0,60,517,1.0,2.0,0.0,1.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00
6,2021/02/11,05:30:00,192,78,WB,Emerald Dr to 78 WB,4.285,3.9,69,578,NaN,NaN,0.0,0.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25225,2021/02/11,19:30:00,23004,78,WB,78 WB Connector from 15 NB,16.270,4.0,57,487,NaN,NaN,NaN,NaN,NaN,0xF8,1900-01-01 19:30:00,1900-01-01 19:31:00
25226,2021/02/11,19:30:00,33004,78,WB,78 WB Connector from 15 SB,16.272,4.0,57,487,NaN,NaN,NaN,NaN,NaN,0xF8,1900-01-01 19:30:00,1900-01-01 19:31:00
25227,2021/02/11,19:30:00,239,78,WB,Centre City Pkwy SB to 78 WB,17.283,2.9,54,335,NaN,NaN,1.0,3.0,NaN,0xE0,1900-01-01 19:30:00,1900-01-01 19:31:00
25228,2021/02/11,19:30:00,240,78,WB,Centre City Pkwy NB to 78 WB,17.340,0.0,0,0,NaN,NaN,0.0,3.0,NaN,0x00,1900-01-01 19:30:00,1900-01-01 19:31:00


In [14]:
eb["eb_convtime"] = eb['Time'].apply(convtime)

/Users/vincewong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
def name_transform(x):
    if 'WB' in x:
        x = x.replace('WB','EB')
    return x

In [16]:
wb['transform_name'] = wb['Name'].apply(name_transform)

In [17]:
eb.head(20)

,Date,Time,Id,Route,Direction,Name,MlDetPostmile,pcw1occ,pcw1speed,pcw1vol,r_1_offcnt,r_2_offcnt,Psg 1 Cnt,Psg 2 Cnt,Psg 3 Cnt,Mainline Lanes,eb_convtime
1,2021/02/11,05:30:00,201,78,EB,Jefferson St to 78 EB,0.8580,1.4,73,213,0.0,0.0,0.0,0.0,0.0,0xE0,1900-01-01 05:30:00
2,2021/02/11,05:30:00,200,78,EB,El Camino Real to 78 EB,1.5940,2.5,63,335,1.0,NaN,3.0,3.0,NaN,0xE0,1900-01-01 05:30:00
4,2021/02/11,05:30:00,199,78,EB,Plaza Dr to 78 EB,3.5860,2.1,59,274,0.0,0.0,0.0,0.0,NaN,0xFC,1900-01-01 05:30:00
5,2021/02/11,05:30:00,24,78,EB,Emerald Dr to 78 EB,4.4740,3.0,52,335,0.0,0.0,0.0,0.0,NaN,0xFC,1900-01-01 05:30:00
6,2021/02/11,05:30:00,205,78,EB,Vista Village Dr to 78 EB,6.3200,3.0,79,517,0.0,0.0,0.0,4.0,1.0,0xE0,1900-01-01 05:30:00
7,2021/02/11,05:30:00,198,78,EB,Escondido Ave to 78 EB,6.8860,3.3,78,548,0.0,1.0,2.0,4.0,NaN,0xE0,1900-01-01 05:30:00
8,2021/02/11,05:30:00,197,78,EB,Mar Vista Dr to 78 EB,7.7960,5.6,71,853,0.0,NaN,1.0,0.0,NaN,0xE0,1900-01-01 05:30:00
9,2021/02/11,05:30:00,203,78,EB,Sycamore Ave to 78 EB,9.2180,4.1,70,609,8.0,0.0,0.0,2.0,2.0,0xE0,1900-01-01 05:30:00
10,2021/02/11,05:30:00,152,78,EB,Rho Santa Fe Rd to 78 EB,10.6800,4.6,65,639,1.0,1.0,0.0,0.0,NaN,0xE0,1900-01-01 05:30:00
11,2021/02/11,05:30:00,398,78,EB,Las Posas Rd/Grand Ave to 78 EB,11.3640,4.4,87,822,0.0,1.0,0.0,3.0,NaN,0xE0,1900-01-01 05:30:00


In [18]:
wb.head()

,Date,Time,Id,Route,Direction,Name,MlDetPostmile,pcw1occ,pcw1speed,pcw1vol,r_1_offcnt,r_2_offcnt,Psg 1 Cnt,Psg 2 Cnt,Psg 3 Cnt,Mainline Lanes,wb_convtime,delay_time,transform_name
1,2021/02/11,05:30:00,190,78,WB,Jefferson St to 78 WB,0.653,3.6,64,487,0.0,1.0,0.0,0.0,0.0,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Jefferson St to 78 EB
2,2021/02/11,05:30:00,191,78,WB,El Camino Real to 78 WB,1.387,3.8,64,517,0.0,1.0,1.0,2.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,El Camino Real to 78 EB
3,2021/02/11,05:30:00,66,78,WB,College Blvd SB to 78 WB,3.245,3.7,54,426,NaN,NaN,0.0,2.0,0.0,0xFC,1900-01-01 05:30:00,1900-01-01 05:31:00,College Blvd SB to 78 EB
4,2021/02/11,05:30:00,270,78,WB,College Blvd NB to 78 WB,3.330,4.0,60,517,1.0,2.0,0.0,1.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,College Blvd NB to 78 EB
6,2021/02/11,05:30:00,192,78,WB,Emerald Dr to 78 WB,4.285,3.9,69,578,NaN,NaN,0.0,0.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Emerald Dr to 78 EB


In [19]:
new_df = pd.merge(eb, wb,  how='inner', left_on=['Name','eb_convtime'], right_on = ['transform_name','delay_time'])

In [20]:
pd.set_option('display.max_columns', None)

In [21]:
new_df.head(30)

,Date_x,Time_x,Id_x,Route_x,Direction_x,Name_x,MlDetPostmile_x,pcw1occ_x,pcw1speed_x,pcw1vol_x,r_1_offcnt_x,r_2_offcnt_x,Psg 1 Cnt_x,Psg 2 Cnt_x,Psg 3 Cnt_x,Mainline Lanes_x,eb_convtime,Date_y,Time_y,Id_y,Route_y,Direction_y,Name_y,MlDetPostmile_y,pcw1occ_y,pcw1speed_y,pcw1vol_y,r_1_offcnt_y,r_2_offcnt_y,Psg 1 Cnt_y,Psg 2 Cnt_y,Psg 3 Cnt_y,Mainline Lanes_y,wb_convtime,delay_time,transform_name
0,2021/02/11,05:31:00,201,78,EB,Jefferson St to 78 EB,0.858,1.1,75,182,0.0,1.0,0.0,0.0,0.0,0xE0,1900-01-01 05:31:00,2021/02/11,05:30:00,190,78,WB,Jefferson St to 78 WB,0.653,3.6,64,487,0.0,1.0,0.0,0.0,0.0,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Jefferson St to 78 EB
1,2021/02/11,05:31:00,200,78,EB,El Camino Real to 78 EB,1.594,1.7,65,243,0.0,NaN,2.0,2.0,NaN,0xE0,1900-01-01 05:31:00,2021/02/11,05:30:00,191,78,WB,El Camino Real to 78 WB,1.387,3.8,64,517,0.0,1.0,1.0,2.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,El Camino Real to 78 EB
2,2021/02/11,05:31:00,24,78,EB,Emerald Dr to 78 EB,4.474,2.7,70,396,0.0,1.0,2.0,1.0,NaN,0xFC,1900-01-01 05:31:00,2021/02/11,05:30:00,192,78,WB,Emerald Dr to 78 WB,4.285,3.9,69,578,NaN,NaN,0.0,0.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Emerald Dr to 78 EB
3,2021/02/11,05:31:00,205,78,EB,Vista Village Dr to 78 EB,6.320,2.4,76,396,1.0,0.0,0.0,3.0,2.0,0xE0,1900-01-01 05:31:00,2021/02/11,05:30:00,204,78,WB,Vista Village Dr to 78 WB,6.141,1.6,81,274,0.0,0.0,0.0,2.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Vista Village Dr to 78 EB
4,2021/02/11,05:31:00,197,78,EB,Mar Vista Dr to 78 EB,7.796,5.0,77,822,1.0,NaN,0.0,1.0,NaN,0xE0,1900-01-01 05:31:00,2021/02/11,05:30:00,195,78,WB,Mar Vista Dr to 78 WB,7.763,2.3,75,365,1.0,NaN,0.0,0.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Mar Vista Dr to 78 EB
5,2021/02/11,05:31:00,203,78,EB,Sycamore Ave to 78 EB,9.218,4.2,74,670,2.0,0.0,0.0,0.0,1.0,0xE0,1900-01-01 05:31:00,2021/02/11,05:30:00,202,78,WB,Sycamore Ave to 78 WB,8.973,2.5,86,457,3.0,4.0,0.0,1.0,2.0,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Sycamore Ave to 78 EB
6,2021/02/11,05:31:00,152,78,EB,Rho Santa Fe Rd to 78 EB,10.680,5.7,62,761,0.0,1.0,0.0,4.0,NaN,0xE0,1900-01-01 05:31:00,2021/02/11,05:30:00,196,78,WB,Rho Santa Fe Rd to 78 WB,10.439,4.0,78,670,3.0,2.0,0.0,1.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Rho Santa Fe Rd to 78 EB
7,2021/02/11,05:31:00,236,78,EB,Barham/Woodland to 78 EB,14.860,3.6,64,487,NaN,NaN,0.0,2.0,NaN,0xFC,1900-01-01 05:31:00,2021/02/11,05:30:00,235,78,WB,Barham/Woodland to 78 WB,14.140,6.7,61,883,1.0,NaN,0.0,2.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Barham/Woodland to 78 EB
8,2021/02/11,05:31:00,13006,78,EB,Nordahl Rd to 78 EB,15.596,5.0,66,700,0.0,0.0,2.0,2.0,NaN,0xE0,1900-01-01 05:31:00,2021/02/11,05:30:00,13007,78,WB,Nordahl Rd to 78 WB,15.370,6.9,65,975,1.0,4.0,0.0,2.0,NaN,0xE0,1900-01-01 05:30:00,1900-01-01 05:31:00,Nordahl Rd to 78 EB
9,2021/02/11,05:32:00,201,78,EB,Jefferson St to 78 EB,0.858,1.6,69,243,0.0,0.0,0.0,0.0,0.0,0xE0,1900-01-01 05:32:00,2021/02/11,05:31:00,190,78,WB,Jefferson St to 78 WB,0.653,4.1,65,578,0.0,1.0,0.0,0.0,0.0,0xE0,1900-01-01 05:31:00,1900-01-01 05:32:00,Jefferson St to 78 EB


In [22]:
new_df = new_df[['Name_x', 'transform_name','pcw1occ_x', 'pcw1speed_x', 'pcw1vol_x', 'r_1_offcnt_x', 'r_2_offcnt_x',
                'pcw1occ_y', 'pcw1speed_y', 'pcw1vol_y', 'Psg 1 Cnt_x', 
                 'Psg 2 Cnt_x', 'Psg 3 Cnt_x','r_1_offcnt_y', 'r_2_offcnt_y']]

In [23]:
new_df

,Name_x,transform_name,pcw1occ_x,pcw1speed_x,pcw1vol_x,r_1_offcnt_x,r_2_offcnt_x,pcw1occ_y,pcw1speed_y,pcw1vol_y,Psg 1 Cnt_x,Psg 2 Cnt_x,Psg 3 Cnt_x,r_1_offcnt_y,r_2_offcnt_y
0,Jefferson St to 78 EB,Jefferson St to 78 EB,1.1,75,182,0.0,1.0,3.6,64,487,0.0,0.0,0.0,0.0,1.0
1,El Camino Real to 78 EB,El Camino Real to 78 EB,1.7,65,243,0.0,NaN,3.8,64,517,2.0,2.0,NaN,0.0,1.0
2,Emerald Dr to 78 EB,Emerald Dr to 78 EB,2.7,70,396,0.0,1.0,3.9,69,578,2.0,1.0,NaN,NaN,NaN
3,Vista Village Dr to 78 EB,Vista Village Dr to 78 EB,2.4,76,396,1.0,0.0,1.6,81,274,0.0,3.0,2.0,0.0,0.0
4,Mar Vista Dr to 78 EB,Mar Vista Dr to 78 EB,5.0,77,822,1.0,NaN,2.3,75,365,0.0,1.0,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7555,Mar Vista Dr to 78 EB,Mar Vista Dr to 78 EB,4.1,76,670,3.0,NaN,7.2,71,1096,0.0,3.0,NaN,0.0,NaN
7556,Sycamore Ave to 78 EB,Sycamore Ave to 78 EB,4.7,78,792,2.0,4.0,4.2,87,792,0.0,3.0,3.0,2.0,3.0
7557,Rho Santa Fe Rd to 78 EB,Rho Santa Fe Rd to 78 EB,5.5,65,761,1.0,3.0,4.6,80,792,0.0,2.0,NaN,1.0,2.0
7558,Barham/Woodland to 78 EB,Barham/Woodland to 78 EB,5.9,67,853,NaN,NaN,8.0,59,1005,0.0,1.0,NaN,3.0,NaN


# Calculations

In [24]:
# I think vol_x is upstream and y is downstream

# For East Bound (variables with x):

density_downstream = []
for i in range(len(new_df['pcw1vol_x'])):
    if new_df['pcw1vol_x'][i] != 0:
        nds = (1/new_df['pcw1vol_x'][i]) * (new_df['pcw1speed_x'][i] + new_df['r_1_offcnt_x'][i])
        # assuming r_1_offcnt_x is ramp, what is ramp speed?
    density_downstream.append(nds)

In [25]:
density_downstream

[0.4120879120879121,
 0.2674897119341564,
 0.1767676767676768,
 0.19444444444444445,
 0.09489051094890512,
 0.11343283582089553,
 0.08147174770039423,
 nan,
 0.09428571428571429,
 0.28395061728395066,
 0.3351648351648352,
 0.17313432835820897,
 0.14788732394366197,
 0.12151067323481116,
 0.14285714285714285,
 0.08712121212121213,
 nan,
 0.13963039014373718,
 0.25,
 0.2469135802469136,
 0.1917808219178082,
 0.15258215962441316,
 0.09217171717171718,
 0.16605839416058393,
 0.10954616588419405,
 nan,
 0.09523809523809523,
 0.16712328767123288,
 0.1492537313432836,
 0.21052631578947367,
 0.13686131386861314,
 0.13686131386861314,
 0.17794970986460348,
 0.12591240875912407,
 nan,
 0.08428571428571428,
 0.2757201646090535,
 0.1373134328358209,
 0.23722627737226276,
 0.21643835616438356,
 0.07035647279549719,
 0.12989045383411582,
 0.1411992263056093,
 nan,
 0.08857142857142856,
 0.2674897119341564,
 0.17105263157894735,
 0.24817518248175183,
 0.22686567164179106,
 0.08720271800679502,
 0.090

In [26]:
new_df.head()

,Name_x,transform_name,pcw1occ_x,pcw1speed_x,pcw1vol_x,r_1_offcnt_x,r_2_offcnt_x,pcw1occ_y,pcw1speed_y,pcw1vol_y,Psg 1 Cnt_x,Psg 2 Cnt_x,Psg 3 Cnt_x,r_1_offcnt_y,r_2_offcnt_y
0,Jefferson St to 78 EB,Jefferson St to 78 EB,1.1,75,182,0.0,1.0,3.6,64,487,0.0,0.0,0.0,0.0,1.0
1,El Camino Real to 78 EB,El Camino Real to 78 EB,1.7,65,243,0.0,NaN,3.8,64,517,2.0,2.0,NaN,0.0,1.0
2,Emerald Dr to 78 EB,Emerald Dr to 78 EB,2.7,70,396,0.0,1.0,3.9,69,578,2.0,1.0,NaN,NaN,NaN
3,Vista Village Dr to 78 EB,Vista Village Dr to 78 EB,2.4,76,396,1.0,0.0,1.6,81,274,0.0,3.0,2.0,0.0,0.0
4,Mar Vista Dr to 78 EB,Mar Vista Dr to 78 EB,5.0,77,822,1.0,NaN,2.3,75,365,0.0,1.0,NaN,1.0,NaN


In [27]:
new_df['Psg 1 Cnt_x'] = new_df['Psg 1 Cnt_x'].fillna(0)
new_df['Psg 2 Cnt_x'] = new_df['Psg 2 Cnt_x'].fillna(0)
new_df['Psg 3 Cnt_x'] = new_df['Psg 3 Cnt_x'].fillna(0)
new_df['r_1_offcnt_x'] = new_df['r_1_offcnt_x'].fillna(0)
new_df['r_2_offcnt_x'] = new_df['r_2_offcnt_x'].fillna(0)

In [28]:
new = pd.DataFrame()

In [39]:
diff_df= pd.DataFrame()

In [34]:
vmergelst=["vmerge_2016",
"vmerge_1942",
"vmerge_1868",
"vmerge_1792",
"vmerge_1718",
"vmerge_1644",
"vmerge_1570",
"vmerge_1496",
"vmerge_1420",
"vmerge_1346",
"vmerge_1272",
"vmerge_1198",
"vmerge_1124",
"vmerge_1048",
"vmerge_968"]

In [50]:
row1 = new_df.iloc[[0]]
row1['dens_ups']= row1['pcw1speed_x']/row1['pcw1vol_x']
row1['q_ramp'] = (row1['Psg 1 Cnt_x']+row1['Psg 2 Cnt_x']+row1['Psg 3 Cnt_x'])*120
row1['q_exit'] = (row1['r_1_offcnt_x']+row1['r_2_offcnt_x'])*120
rates = [1008,971,934,896,859,822,785, 748,710,673,636,599,562,524,484]
rates2 = []
# double the rate for two ramps
for i in rates:
    rates2.append(i*2)
# density down stream for each rate
for i in rates2:
    vols = (((5280*3)*row1['pcw1vol_x'])/((1.75*1.47*3*row1['pcw1speed_x'])+i))-(14.75-(1.75*1.47))
    row1['vmerge_'+str(i)] = vols
    row1['nds_'+str(i)] = (1/row1['pcw1vol_x']) * (row1['pcw1speed_x'] + (i/3))-((.33* row1['q_exit'])/row1['vmerge_'+str(i)])
# take the minimum of upstream volume and eqn
for i in vmergelst:
    row1[i] = row1[['pcw1vol_x',i]].min(axis=1)
# for i in rates2:
#     row1['nds_'+str(i)] = (1/row1['pcw1vol_x']) * (row1['pcw1speed_x'] + (i/3))-((.33* row1['q_exit'])/row1['vmerge_'+str(i)])
for i in rates2:
    diff_df['diff_'+str(i)]=row1['nds_'+str(i)]-(row1['dens_ups']*1.15)
meanloss = []
for i in diff_df.columns:
    meanloss.append(np.mean(diff_df[i]))
best_rate = diff_df.min(axis=0).idxmin()
best_rate = int(best_rate[5:])
best_rate

/Users/vincewong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/vincewong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/vincewong/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFr

968

In [ ]:
# for i in range(new_df.shape[0]):
#     row1 = new_df.iloc[[i]]
#     row1['dens_ups']= row1['pcw1speed_x']/row1['pcw1vol_x']
#     row1['q_ramp'] = (row1['Psg 1 Cnt_x']+row1['Psg 2 Cnt_x']+row1['Psg 3 Cnt_x'])*120
#     row1['q_exit'] = (row1['r_1_offcnt_x']+row1['r_2_offcnt_x'])*120
#     rates = [1008,971,934,896,859,822,785, 748,710,673,636,599,562,524,484]
#     rates2 = []
#     for i in rates:
#         rates2.append(i*2)
#     for i in rates2:
#         vols = (((5280*3)*row1['pcw1vol_x'])/((1.75*1.47*3*row1['pcw1speed_x'])+i))-(14.75-(1.75*1.47))
#         row1['vmerge_'+str(i)] = vols
#         row1['nds_'+str(i)] = (1/row1['pcw1vol_x']) * (row1['pcw1speed_x'] + (i/3))-(.33* row1['q_exit'])
#     for i in vmergelst:
#         row1[i] = row1[['pcw1vol_x',i]].min(axis=1)
#     for i in rates2:
#         row1['nds_'+str(i)] = (1/row1['pcw1vol_x']) * (row1['pcw1speed_x'] + (i/3))-((.33* row1['q_exit'])/row1['vmerge_'+str(i)])
#     for i in rates2:
#         diff_df['diff_'+str(i)]=row1['nds_'+str(i)]-(row1['dens_ups']*1.15)
#     meanloss = []
#     diff_df
#     #best_rate = min(meanloss)
#     for i in diff_df.columns:
#         meanloss.append(np.mean(diff_df[i]))

In [46]:
diff_df.min(axis=0).idxmin()

'diff_968'

In [38]:
meanloss

[3.412912087912088,
 3.2773809523809527,
 3.141849816849817,
 3.002655677655678,
 2.867124542124542,
 2.7315934065934067,
 2.596062271062271,
 2.460531135531136,
 2.321336996336996,
 2.185805860805861,
 2.050274725274725,
 1.9147435897435898,
 1.7792124542124543,
 1.6400183150183152,
 1.4934981684981687]

In [30]:
# new_df['dens_ups']= new_df['pcw1speed_x']/new_df['pcw1vol_x']

In [31]:
# new_df.head()

In [32]:
# new_df['q_ramp'] = (new_df['Psg 1 Cnt_x']+new_df['Psg 2 Cnt_x']+new_df['Psg 3 Cnt_x'])*120
# new_df['q_exit'] = (new_df['r_1_offcnt_x']+new_df['r_2_offcnt_x'])*120

In [35]:
#new_df['q_ramp']<new_df['q_exit']

In [ ]:
# rates = [1008,971,934,896,859,822,785, 748,710,673,636,599,562,524,484]
# rates2 = []
# for i in rates:
#     rates2.append(i*2)

In [ ]:
# for i in rates2:
#     vols = (((5280*3)*new_df['pcw1vol_x'])/((1.75*1.47*3*new_df['pcw1speed_x'])+i))-(14.75-(1.75*1.47))
#     new_df['vmerge_'+str(i)] = vols
   
#     #new_df['nds_'+str(i)] = (1/new_df['pcw1vol_x']) * (new_df['pcw1speed_x'] + (i/3))-(.33* new_df['q_exit'])

In [ ]:
# for i in vmergelst:
#     new_df[i] = new_df[['pcw1vol_x',i]].min(axis=1)

In [ ]:
# for i in rates2:
#     new_df['nds_'+str(i)] = (1/new_df['pcw1vol_x']) * (new_df['pcw1speed_x'] + (i/3))-((.33* new_df['q_exit'])/new_df['vmerge_'+str(i)])

In [ ]:
# new_df.head()

In [ ]:
# for i in rates2:
#     diff_df['diff_'+str(i)]=new_df['nds_'+str(i)]-(new_df['dens_ups']*1.15)

In [ ]:
# meanloss = []
# for i in diff_df.columns:
#     meanloss.append(np.mean(diff_df[i]))

In [51]:
diff_df.head()

""


In [ ]:
diffs = pd.DataFrame()
diffs['rates'] = rates2
diffs['loss'] = meanloss
diffs

## Normal Distribution Generated Rates

In [ ]:
# mu, sigma = 968, 1 # mean and standard deviation
# new_rates = np.random.normal(mu, sigma, 100)
# new_rates

In [ ]:
# new_diff=pd.DataFrame()
# for i in new_rates:
#     new_diff['diff_'+str(i)]=new_df['nds_'+str(i)]-(new_df['dens_ups']*1.15)

In [ ]:
# for i in new_rates:
#     new_vols = (((5280*3)*new_df['pcw1vol_x'])/((1.75*1.47*3*new_df['pcw1speed_x'])+i))-(14.75-(1.75*1.47))
#     new_df['vmerge_'+str(i)] = new_vols

In [ ]:
# gen = new_df.iloc[:,-100:]

In [ ]:
# gencols = gen.columns

In [ ]:
# gen['pcw1vol_x'] = new_df['pcw1vol_x']
# gen['pcw1speed_x'] = new_df['pcw1speed_x']
# gen['q_exit'] = new_df['q_exit']
# gen['q_exit'] = new_df['dens_ups']

In [ ]:
# gen.describe()

In [ ]:
# for i in gencols:
#     gen[i] = gen[['pcw1vol_x',i]].min(axis=1)

In [ ]:
# for i in new_rates:
#     gen['nds_'+str(i)] = (1/new_df['pcw1vol_x']) * (new_df['pcw1speed_x'] + (i/3))-((.33* new_df['q_exit'])/new_df['vmerge_'+str(i)])

In [ ]:
# gen.describe()

In [ ]:
# import statistics 

In [ ]:
# gen_diff = pd.DataFrame()

In [ ]:
# for i in new_rates:
#     gen_diff['diff_'+str(i)]=gen['nds_'+str(i)]-(new_df['dens_ups']*1.15)

In [ ]:
# gen_diff

In [ ]:
# gen_diff.describe()

In [ ]:
# gen_meanloss = []
# for i in gen_diff.columns:
#     gen_meanloss.append(np.mean(gen_diff[i]))

In [ ]:
# gendiffs = pd.DataFrame()
# gendiffs['rates'] = new_rates
# gendiffs['loss'] = gen_meanloss
# gendiffs

In [ ]:
# gendiffs['loss'].min()

In [ ]:
# filtered_df = new_df[new_df[['Name_x', 'transform_name']].notnull()] #this makes all values NaN?

In [ ]:
# filtered_df

In [ ]:
# q_exit = (filtered_df.r_1_offcnt_x+filtered_df.r_1_offcnt_y)*120
# q_exit.unique()